<a href="https://colab.research.google.com/github/Allenchou0708/triplelearning-for-image-retargeting/blob/main/triplelearning_for_image_retargeting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Matting
!pip install rembg onnxruntime matplotlib pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.5 MB/s eta 0:00:00


In [2]:
from rembg import remove
from PIL import Image, ImageChops, ImageOps
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import numpy as np

In [3]:

def extract_background(original, foreground):
    original_np = np.array(original)
    foreground_np = np.array(foreground)

    alpha = foreground_np[:, :, 3] / 255.0
    background_np = original_np.copy()
    for c in range(3):
        background_np[:, :, c] = background_np[:, :, c] * (1 - alpha)
    background_np[:, :, 3] = 255
    return Image.fromarray(background_np.astype('uint8'), 'RGBA')


# 顯示原圖、前景與背景
def draw_ofb_graph(original_img, foreground_img, background_img):
  fig, axes = plt.subplots(1, 3, figsize=(18, 6))
  axes[0].imshow(original_img)
  axes[0].axis("off")

  axes[1].imshow(foreground_img)
  axes[1].axis("off")

  axes[2].imshow(background_img)
  axes[2].axis("off")

  plt.tight_layout()
  plt.show()


In [4]:

def matting(original_img_name):

  original_img = Image.open(original_img_name).convert("RGBA")

  # 使用 rembg 分離前景 (matting)
  foreground_img = remove(original_img)

  background_img = extract_background(original_img, foreground_img)

  return foreground_img, background_img



In [5]:
!wget https://people.csail.mit.edu/mrub/retargetme/download/images-20100824.zip

--2025-07-01 10:54:07--  https://people.csail.mit.edu/mrub/retargetme/download/images-20100824.zip
Resolving people.csail.mit.edu (people.csail.mit.edu)... 128.52.131.233
Connecting to people.csail.mit.edu (people.csail.mit.edu)|128.52.131.233|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 61460154 (59M) [application/zip]
Saving to: ‘images-20100824.zip’

images-20100824.zip 100%[===================>]  58.61M  14.1MB/s    in 5.5s    

2025-07-01 10:54:14 (10.7 MB/s) - ‘images-20100824.zip’ saved [61460154/61460154]



In [6]:
!unzip -q images-20100824.zip -d "./images"

In [7]:
import glob
import os

image_paths = glob.glob('./images/*.png')
print(f"# of images : {len(image_paths)}")
# 列出所有符合該條件的路徑 ex ./360-Dataset/360-Dataset/images/image1870.jpg

# of images : 80


In [8]:

for idx, image in enumerate(image_paths) :
  foreground_img, background_img = matting(image)
  original_img = Image.open(image).convert("RGBA")
  draw_ofb_graph(original_img, foreground_img, background_img)
  # if idx > 5 :
  #   break

 18%|██████▉                               | 32.3M/176M [00:00<00:01, 80.6MB/s]

ChunkedEncodingError: ('Connection broken: IncompleteRead(33215841 bytes read, 142781800 more expected)', IncompleteRead(33215841 bytes read, 142781800 more expected))

In [ ]:
# Option 2: use with BiRefNet

!git clone https://github.com/ZhengPeng7/BiRefNet

In [ ]:
!pip install timm
!pip install kornia
!pip install einops
!pip install prettytable
!pip install tabulate
!pip install accelerate
!pip install huggingface-hub>0.25

In [ ]:
from transformers import AutoModelForImageSegmentation
birefnet = AutoModelForImageSegmentation.from_pretrained('zhengpeng7/BiRefNet', trust_remote_code=True)

In [ ]:
# ZhengPeng7/BiRefNet

# Imports
from PIL import Image
import matplotlib.pyplot as plt
import torch
from torchvision import transforms

torch.set_float32_matmul_precision(['high', 'highest'][0])
birefnet.to('cuda')
birefnet.eval()
birefnet.half() #用 16 倍精度

def extract_object(birefnet, imagepath):
    # Data settings
    image_size = (1024, 1024)
    transform_image = transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    image = Image.open(imagepath)
    input_images = transform_image(image).unsqueeze(0).to('cuda').half()

    # Prediction
    with torch.no_grad():
        preds = birefnet(input_images)[-1].sigmoid().cpu()
    pred = preds[0].squeeze()
    pred_pil = transforms.ToPILImage()(pred)
    mask = pred_pil.resize(image.size)
    image.putalpha(mask)
    return image, mask

# Visualization
plt.axis("off")
plt.imshow(extract_object(birefnet, imagepath="./images/ArtRoom.png")[0])
plt.show()
